# Results Analysis - Use Features and Similarity Measure 

In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir('../')

In [2]:
from scipy.signal import argrelmin, argrelmax

In [3]:
from dap import DAPcython
from dap.utils import obs_params, load_current
from tqdm import tqdm
from scipy.spatial import distance

### Set Parameters

In [4]:
dt = 1e-2
params, labels = obs_params(reduced_model=False)
data_dir = '/home/alteska/Desktop/LFI_DAP/data/rawData/2015_08_26b.dat'

### load the file

In [5]:
df_param = pd.read_csv('dap_models_4_param_1x20.csv')

### calculate DAP

In [6]:
# load the input data
Ir, vr, tr, t_onr, t_offr, dtr = load_current(data_dir, protocol='rampIV', ramp_amp=3.1)
Is, vs, ts, t_ons, t_offs, dts = load_current(data_dir, protocol='IV', ramp_amp=1)

# define a model
dap = DAPcython(-75, params)

In [7]:
# run models on original parameters
U_step = dap.simulate(dts, ts, Is)
U_ramp = dap.simulate(dtr, tr, Ir)

### calculate the similarities

In [8]:
d_step = distance.euclidean(vs, U_step)
d_step

2657.835169092602

In [9]:
d_ramp = distance.euclidean(vr, U_ramp)
d_ramp

503.9336667145733

In [21]:
d_step+d_ramp

3161.7688358071755

### run for all cells and save into the the DF

In [11]:
df_paramT = df_param.transpose()
df_paramT.head()

,0,1,2,3
Unnamed: 0,gbar_nap,gbar_leak,gbar_nat,gbar_kdr
1x14,17.1737,1.07467,131.423,3.41051
1x35,17.0972,1.03987,117.684,3.73748
1x9,19.0074,0.991425,95.3657,2.97607
1x19,16.3705,1.0285,141.775,3.43003


In [12]:
df_paramT.drop('Unnamed: 0', inplace=True)
df_paramT.head()

,0,1,2,3
1x14,17.1737,1.07467,131.423,3.41051
1x35,17.0972,1.03987,117.684,3.73748
1x9,19.0074,0.991425,95.3657,2.97607
1x19,16.3705,1.0285,141.775,3.43003
1x24,15.1533,1.10018,163.721,3.05554


In [13]:
daps = []
U_steps = []
U_ramps = []

for i, j in tqdm(df_paramT.iterrows()):
    # get parameters
    par_temp = j.values

    # define a model
    daps.append(DAPcython(-75, j))

    # run model
    U_steps.append(dap.simulate(dts, ts, Is).transpose()[0])
    U_ramps.append(dap.simulate(dtr, tr, Ir).transpose()[0])

34it [00:38,  1.12s/it]


### Create DF with traces

In [14]:
df_arr = pd.DataFrame({'traces': U_steps})
df_arr.set_index(df_paramT.index.values, inplace=True)
df_arr.head()

,traces
1x14,"[-74.9967136606412, -74.99173438388436, -74.98..."
1x35,"[-74.99369855519795, -74.9801286049812, -75.00..."
1x9,"[-74.99252197828939, -75.01928528271377, -75.0..."
1x19,"[-75.02820243848119, -75.00859023673239, -75.0..."
1x24,"[-74.99027450234452, -74.98855592429483, -74.9..."


In [15]:
result = pd.merge(
    df_paramT,
    df_arr,
    how='left',
    left_index=True, # Merge on both indexes, since right only has 0...
    right_index=True # all the other rows will be NaN
)

result

,0,1,2,3,traces
1x14,17.1737,1.07467,131.423,3.41051,"[-74.9967136606412, -74.99173438388436, -74.98..."
1x35,17.0972,1.03987,117.684,3.73748,"[-74.99369855519795, -74.9801286049812, -75.00..."
1x9,19.0074,0.991425,95.3657,2.97607,"[-74.99252197828939, -75.01928528271377, -75.0..."
1x19,16.3705,1.0285,141.775,3.43003,"[-75.02820243848119, -75.00859023673239, -75.0..."
1x24,15.1533,1.10018,163.721,3.05554,"[-74.99027450234452, -74.98855592429483, -74.9..."
1x16,16.5159,1.12175,120.139,3.41498,"[-74.98312940284475, -75.00864659084881, -75.0..."
1x31k,14.3871,1.09387,164.15,3.12817,"[-75.0006431476699, -74.99026032405145, -75.02..."
1x12,16.1834,1.03587,108.917,2.99428,"[-74.99738778559906, -74.98086021896435, -75.0..."
1x27,16.5867,1.04691,169.626,3.17433,"[-74.99671906923204, -74.9954847010249, -75.00..."
1x23,18.9342,1.02568,115.465,3.19455,"[-75.01263693828218, -74.99811865924114, -75.0..."


In [24]:
result = pd.merge(
    df_paramT,
    df_arr,
    how='left',
    left_index=True, # Merge on both indexes, since right only has 0...
    right_index=True # all the other rows will be NaN
)

result

,0,1,2,3,traces
1x14,17.1737,1.07467,131.423,3.41051,"[-74.9967136606412, -74.99173438388436, -74.98..."
1x35,17.0972,1.03987,117.684,3.73748,"[-74.99369855519795, -74.9801286049812, -75.00..."
1x9,19.0074,0.991425,95.3657,2.97607,"[-74.99252197828939, -75.01928528271377, -75.0..."
1x19,16.3705,1.0285,141.775,3.43003,"[-75.02820243848119, -75.00859023673239, -75.0..."
1x24,15.1533,1.10018,163.721,3.05554,"[-74.99027450234452, -74.98855592429483, -74.9..."
1x16,16.5159,1.12175,120.139,3.41498,"[-74.98312940284475, -75.00864659084881, -75.0..."
1x31k,14.3871,1.09387,164.15,3.12817,"[-75.0006431476699, -74.99026032405145, -75.02..."
1x12,16.1834,1.03587,108.917,2.99428,"[-74.99738778559906, -74.98086021896435, -75.0..."
1x27,16.5867,1.04691,169.626,3.17433,"[-74.99671906923204, -74.9954847010249, -75.00..."
1x23,18.9342,1.02568,115.465,3.19455,"[-75.01263693828218, -74.99811865924114, -75.0..."


In [25]:
result2 = pd.merge(
    result,
    df_arr,
    how='left',
    left_index=True, # Merge on both indexes, since right only has 0...
    right_index=True # all the other rows will be NaN
)

result2

,0,1,2,3,traces_x,traces_y
1x14,17.1737,1.07467,131.423,3.41051,"[-74.9967136606412, -74.99173438388436, -74.98...","[-74.9967136606412, -74.99173438388436, -74.98..."
1x35,17.0972,1.03987,117.684,3.73748,"[-74.99369855519795, -74.9801286049812, -75.00...","[-74.99369855519795, -74.9801286049812, -75.00..."
1x9,19.0074,0.991425,95.3657,2.97607,"[-74.99252197828939, -75.01928528271377, -75.0...","[-74.99252197828939, -75.01928528271377, -75.0..."
1x19,16.3705,1.0285,141.775,3.43003,"[-75.02820243848119, -75.00859023673239, -75.0...","[-75.02820243848119, -75.00859023673239, -75.0..."
1x24,15.1533,1.10018,163.721,3.05554,"[-74.99027450234452, -74.98855592429483, -74.9...","[-74.99027450234452, -74.98855592429483, -74.9..."
1x16,16.5159,1.12175,120.139,3.41498,"[-74.98312940284475, -75.00864659084881, -75.0...","[-74.98312940284475, -75.00864659084881, -75.0..."
1x31k,14.3871,1.09387,164.15,3.12817,"[-75.0006431476699, -74.99026032405145, -75.02...","[-75.0006431476699, -74.99026032405145, -75.02..."
1x12,16.1834,1.03587,108.917,2.99428,"[-74.99738778559906, -74.98086021896435, -75.0...","[-74.99738778559906, -74.98086021896435, -75.0..."
1x27,16.5867,1.04691,169.626,3.17433,"[-74.99671906923204, -74.9954847010249, -75.00...","[-74.99671906923204, -74.9954847010249, -75.00..."
1x23,18.9342,1.02568,115.465,3.19455,"[-75.01263693828218, -74.99811865924114, -75.0...","[-75.01263693828218, -74.99811865924114, -75.0..."


## Specify the Features Function 

In [16]:
def find_spikes(v):
    # look for non-resting potential values
    ind = np.where(v < -10)
    v[ind] = -10
    ind = np.where(np.diff(v) < 0)
    v[ind] = -10

    # look for all slope changes
    v = (np.diff(v) > 0).astype(int)

    # get all change of slopes indices
    v_ind = np.where(np.diff(np.sign(v)))[0]
    
    # return last positive change of slope 
    return v_ind[1::2]


In [17]:
def calc_features_step(U, t, dt, t_on, t_off):
    v = U.copy()
    N = v.shape[0]

    # resting potential
    rest_pot = np.mean(v[t<t_on])
    rest_pot_std = np.std(v[int(.9*t_on/dt):int(t_on/dt)])  
 
    ind = find_spikes(v)

    spike_times = np.array(t)[ind]
    spike_times_stim = spike_times[(spike_times > t_on)  & (spike_times < t_off)]
    ind_stim1 = ind[(spike_times > t_on) & (spike_times < t_off)]
    ind_stim = ind_stim1.astype(int)

    firing_rate = 1e3*np.absolute(spike_times_stim.shape[0]/(t_off-t_on))
    time_1st_spike = spike_times_stim[spike_times_stim>t_on][0]

    ISI = np.diff(spike_times_stim).astype(float)

    
    sum_stats_vec = np.array([
                rest_pot,
                rest_pot_std,
                len(spike_times_stim),
                spike_times_stim,
                firing_rate,
                ISI.mean(),
                ISI.std()
                ])


    return sum_stats_vec

In [18]:
# def flatten(U):
#     v = U.copy()
#     ind = np.where(v < -10)
#     v[ind] = -10
#     ind = np.where(np.diff(v) < 0)
#     v[ind] = -10
    
#     return v

### Ramp Current

In [19]:
def calc_ramp_sum_stats(v, t, dt, t_on, t_off):
    """Calculate summary statistics of a single run with ramp current(single spike with DAP)"""
    stats = []
    stats_idx = []
#     v = v.transpose()
    N = v.shape[0]

    # resting potential
    rest_pot = np.mean(v[t<t_on])
    rest_pot_std = np.std(v[int(.9*t_on/dt):int(t_on/dt)])   # TODO: add if needed

    # RMSE
#     n = len(self.v0)
#     rmse = np.linalg.norm(v - self.v0) / np.sqrt(n)

    # more then one AP:
    multiple_AP = np.shape(np.where(v > 0))[1]

    #case without any action potential or more then one AP
    if (np.all(v <= 20)):
        AP_onsets = 999
        AP_amp = 999
        AP_width = 999
        DAP_amp = 999
        DAP_width = 999
        DAP_deflection = 999
        DAP_time = 999
        mAHP = 999
        fAHP = 999

    else:
        threshold = -30
        # hyperpolarization after DAP
        mAHP_idx = np.argmin(v)
        mAHP = v[mAHP_idx]

        # Action potential
        AP_onsets = np.where(v > threshold)[0]
        AP_start = AP_onsets[0]
        AP_end = AP_onsets[-1]
        AP_max_idx = AP_start + np.argmax(v[AP_start:AP_end])
        AP_max = v[AP_max_idx]
        AP_amp = AP_max - rest_pot

        # AP width
        AP_onsets_half_max = np.where(v > (AP_max+rest_pot)/2)[0]
        AP_width = t[AP_onsets_half_max[-1]] - t[AP_onsets_half_max[0]]

        # DAP: fAHP
        v_dap = v[AP_max_idx:]

        fAHP_idx = argrelmin(v[AP_max_idx:])[0][0] + AP_max_idx
        fAHP = v[fAHP_idx]

        # DAP amplitude
        DAP_max_idx = argrelmax(v_dap)[0][1] + AP_max_idx
        DAP_max = v[DAP_max_idx]
        DAP_amp = DAP_max - rest_pot

        DAP_deflection = DAP_amp - (fAHP - rest_pot)
        DAP_time = t[DAP_max_idx] - t[AP_max_idx]    # Time between AP and DAP maximum

        # Width of DAP: between fAHP and halfsize of fAHP after DAP max
        vnorm = v[DAP_max_idx:] - rest_pot

        if np.any((abs(vnorm) < abs(fAHP - rest_pot)/2)):
            half_max = np.where((abs(vnorm) < abs(fAHP - rest_pot)/2))[0]

            DAP_width_idx = DAP_max_idx + half_max[0]
            DAP_width = (DAP_width_idx - fAHP_idx) * dt
        else:
            DAP_width = 999


    sum_stats_vec = np.array([
                    rest_pot,
                    AP_amp,
                    AP_width,
                    fAHP,
                    DAP_amp,
                    DAP_width,
                    DAP_deflection,
                    DAP_time,
                    mAHP,
                    ])


    return sum_stats_vec

### Test Summary Stats

In [20]:
x0 = calc_features_step(U_steps[0].transpose()[0], ts, dts, t_ons, t_offs)
x0

IndexError: tuple index out of range

In [ ]:
vs

In [ ]:
step_features_labels = ['rest_pot', 'rest_pot_std', 'spike_count', 'spike_times_stim','firing_rate', 'ISI_mean', 'ISI_std']

In [ ]:
x_orig = calc_features_step(vs.transpose(), ts, dts, t_ons, t_offs)
pd.DataFrame(data=x_orig, index=step_features_labels)

In [ ]:
ramp_features_labels = ['rest_pot', 'AP_amp', 'AP_width', 'fAHP', 'DAP_amp', 'DAP_width', 'DAP_deflection','DAP_time', 'mAHP']

In [ ]:
sim = calc_ramp_sum_stats(U_ramps[0], tr, dtr, t_onr, t_offr)
pd.DataFrame(data=sim, index=ramp_features_labels)

### Vizualize found spikes

In [ ]:
t = find_spikes(vs)
x = np.zeros_like(t)

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(t, x, c='r', s=10);
plt.plot(vs);